<a href="https://colab.research.google.com/github/larrytsai50/100-Days-Of-ML-Code/blob/master/Lou/TTS_metadata_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tai5-uan5_gian5-gi2_kang1-ku7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tai5-uan5_gian5-gi2_kang1-ku7: filename=tai5_uan5_gian5_gi2_kang1_ku7-1.1.1-py3-none-any.whl size=134016 sha256=e8d4ed8e13d18ceaac14c9edadf8d5c2889ec73545c95dcd8b90965ef2344186
  Stored in directory: /root/.cache/pip/wheels/b5/32/1b/e687ebc54b3b350f1222115143068c0988df6a88975790f061
Successfully built tai5-uan5_gian5-gi2_kang1-ku7


In [2]:
# 1. 確保您在 /content/ 目錄下
import os
os.chdir('/content/')

# 2. (可選) 移除舊的資料夾以避免衝突
!rm -rf common-tl

# 3. 克隆 GitHub 專案
!git clone https://github.com/IepIweidieng/common-tl.git

# 4. 修正：將專案的根目錄加入 Python 系統路徑
import sys

# 根據專案結構，我們應該加入包含 'phonetic' 模組的目錄。
# 預設情況下，common-tl 專案的結構是：common-tl/src/phonetic/...
# 因此，我們應該加入 common-tl/src
project_path = '/content/common-tl/src'

# 再次檢查確保路徑存在且正確
if not os.path.isdir(project_path):
    # 如果沒有 src 資料夾，則嘗試根目錄
    project_path = '/content/common-tl'

if project_path not in sys.path:
    sys.path.append(project_path)
    print(f"成功將路徑添加到 sys.path: {project_path}")
else:
    print(f"路徑 {project_path} 已存在。")

Cloning into 'common-tl'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 380 (delta 124), reused 143 (delta 88), pack-reused 176 (from 1)
Receiving objects: 100% (380/380), 190.71 KiB | 5.30 MiB/s, done.
Resolving deltas: 100% (229/229), done.
成功將路徑添加到 sys.path: /content/common-tl


In [3]:
from google.colab import drive
import pandas as pd
import re
from 臺灣言語工具.解析整理.拆文分析器 import 拆文分析器
from 臺灣言語工具.音標系統.閩南語.臺灣閩南語羅馬字拼音 import 臺灣閩南語羅馬字拼音
from phonetic.tl import tl_syllable_to_ipa

# 連線到 Google drive
drive.mount('/content/drive')

DATASET_PATH = "/content/drive/MyDrive/台語自然語言模型/Dataset/sui-siann"
# METADATA_OUTPUT_PATH = f"{DATASET_PATH}/metadata"
METADATA_OUTPUT_PATH = "/content/drive/MyDrive/Colab_Notebooks/taiwan_language/dataset/sui-siann"

# 載入資料
df = pd.read_csv(f"{DATASET_PATH}/SuiSiann.csv")

Mounted at /content/drive


In [5]:
import re
import sys
# 假設您已經在 Colab 中成功執行了 git clone 和 sys.path.append
try:
    from phonetic.tl import tl_syllable_to_ipa
except ImportError:
    print("錯誤：無法匯入 common-tl。請先執行 git clone 和 sys.path.append 步驟。")
    sys.exit()

import re
import sys
from typing import List

# ----------------------------------------------------
# 輔助工具和設定
# ----------------------------------------------------

# 聲調變調表：原始聲調 -> 變調後聲調 (使用數字標記，漳州腔範例)
SANDHI_MAP = {
    '1': '7', '2': '1', '3': '2', '4': '8',
    '5': '7', '7': '3', '8': '4', '0': '0'
}

# 台羅拼音轉英文數字拼音
def tailo_num(tailo):
    tail_obj = 拆文分析器.建立句物件(tailo)
    # print(f'tailo_num({tailo}) => tail_obj:<{tail_obj}>'  # larry

    result = tail_obj.轉音(臺灣閩南語羅馬字拼音).看語句()
    # print(f'tail_obj....看語句() => result:<{result}>'  # larry

    english_with_tone_pattern = r'([A-Z][a-zA-Z]*?)'
    cleaned_text = re.sub(english_with_tone_pattern,
                         lambda m: f" <ENG>{m.group(1).lower()}</ENG>",
                         result)

    # 處理特殊符號和標點 (確保每個符號都是獨立 Token)
    cleaned_text = cleaned_text.replace('...', ' ')
    cleaned_text = cleaned_text.replace('---', ' ')
    cleaned_text = cleaned_text.replace('--', ' ')
    cleaned_text = cleaned_text.replace('-', ' ') # 連字符也視為分詞邊界

    return cleaned_text

# 輔助函式：從臺羅音節中分離聲調數字
def split_syllable_and_tone(syllable_str: str) -> tuple[str, str]:
    """
    從音節字串的結尾分離出發音部分和聲調數字。
    假設輸入是標準化後的數字聲調。
    """
    # 匹配結尾的單一數字 (1-8 或 0)
    match = re.search(r'(\d)$', syllable_str)
    if match:
        tone = match.group(1)
        base = syllable_str[:-len(tone)]
        return base, tone

    # 處理特殊符號 (如鼻化符號 ⁿ，但這裡假設輸入已數字化)
    # 如果找不到數字，這裡會導致變調錯誤，但我們假設輸入是正確數字化的臺羅。
    # 這裡假設沒有數字結尾的為基礎調 1 (為了讓程式運行)
    return syllable_str, '1'

# ----------------------------------------------------
# Stage 1 & 2: 英文隔離與文本分詞
# ----------------------------------------------------

def isolate_and_tokenize(text: str) -> List[str]:
    """
    步驟 1: 隔離大寫開頭的英文專有名詞及其聲調數字。
    步驟 2: 清理標點符號和連字符，並將句子分割成 Tokens 列表。
    """
    # **修正 REGEX**: 匹配 [大寫字母開頭的詞彙] + [數字]
    # ([A-Z][a-zA-Z]*) -> Group 1: 英文詞彙
    # (\d)             -> Group 2: 數字聲調
    english_with_tone_pattern = r'([A-Z][a-zA-Z]*?)(\d)'

    # 替換：'Facebook7' -> ' <ENG>Facebook</ENG> 7 ' (將詞彙和數字完全分離)
    cleaned_text = re.sub(english_with_tone_pattern,
                         lambda m: f" <ENG>{m.group(1).lower()}</ENG> {m.group(2)} ",
                         text)

    # 處理特殊符號和標點 (確保每個符號都是獨立 Token)
    cleaned_text = cleaned_text.replace('---', ' ')
    cleaned_text = cleaned_text.replace('--', ' ')
    cleaned_text = cleaned_text.replace('-', ' ') # 連字符也視為分詞邊界

    # 確保常見標點符號被空格隔開
    punctuation_pattern = r'([.,?!:;])'
    cleaned_text = re.sub(punctuation_pattern, r' \1 ', cleaned_text)

    # 清理多餘的空格並分割
    tokens = [t.strip().lower() for t in cleaned_text.split() if t.strip()]

    return tokens

# ----------------------------------------------------
# Stage 3: 連讀變調 (Tone Sandhi)
# ----------------------------------------------------

def apply_sandhi_and_separate(tokens: List[str]) -> List[List[str]]:
    """
    將 Tokens 列表根據標點符號分組，並對每個組內的臺羅音節應用變調。

    輸出格式：[變調後的臺羅音節/標記Tokens, 標點符號, ...]
    """
    final_sandhi_list = []
    current_sandhi_group = []

    # 臺羅音節模式：[小寫字母開頭] + [數字結尾]
    tai_lo_pattern = re.compile(r'^[a-z\u00C0-\u017F]+\d$')

    for part in tokens:
        # 檢查是否為標點符號
        is_punctuation = re.match(r'[.,?!:;]', part)

        # 檢查是否為英文標記或單獨的數字聲調 (這些 Token 不參與變調，但會中斷變調組)
        is_ignored_token = part.startswith('<eng>') or re.fullmatch(r'\d', part)

        if is_punctuation or is_ignored_token:
            # 遇到邊界或不變調的 Token，先處理累積的臺羅組
            if current_sandhi_group:
                # 應用變調規則 (只針對臺羅部分)
                sandhied_syllables = []
                for i, syllable in enumerate(current_sandhi_group):
                    base, original_tone = split_syllable_and_tone(syllable)
                    actual_tone = original_tone

                    # 變調規則：除了組內最後一個音節，所有都變調
                    if i < len(current_sandhi_group) - 1 and original_tone in SANDHI_MAP:
                        actual_tone = SANDHI_MAP[original_tone]

                    sandhied_syllables.append(base + actual_tone)

                final_sandhi_list.extend(sandhied_syllables)
                current_sandhi_group = []

            # 加入當前的標點/忽略 Token
            final_sandhi_list.append(part)

        elif re.fullmatch(tai_lo_pattern, part):
            # 這是臺羅音節，加入當前變調組
            current_sandhi_group.append(part)

        else:
            # 其他未識別的殘留 (例如純數字、純字母)
            final_sandhi_list.append(part)

    # 處理文件末尾可能剩下的變調組
    if current_sandhi_group:
        sandhied_syllables = []
        for i, syllable in enumerate(current_sandhi_group):
            base, original_tone = split_syllable_and_tone(syllable)
            actual_tone = original_tone
            if i < len(current_sandhi_group) - 1 and original_tone in SANDHI_MAP:
                actual_tone = SANDHI_MAP[original_tone]
            sandhied_syllables.append(base + actual_tone)

        final_sandhi_list.extend(sandhied_syllables)

    return final_sandhi_list

# ----------------------------------------------------
# Stage 4: 最終 G2P 轉換與序列輸出
# ----------------------------------------------------
def convert_to_final_ipa(sandhied_tokens: List[str]) -> str:
    """
    將變調後的 tokens 列表轉換為最終的 IPA 序列。
    """
    ipa_sequence = []

    # 臺羅音節模式：[小寫字母] + [數字結尾]
    tai_lo_pattern = re.compile(r'^[a-z\u00C0-\u017F]+\d$')

    for token in sandhied_tokens:
        if token.startswith('<eng>'):
            # 英文標記：直接保留
            ipa_sequence.append(token)

        elif re.fullmatch(tai_lo_pattern, token):
            # 變調後的臺羅音節：執行 G2P 轉換
            try:
                # tl_syllable_to_ipa 處理後會輸出 IPA 符號
                initial_ipa, final_ipa = tl_syllable_to_ipa(token)
                # 臺語 TTS 訓練格式: 將所有音素連起來，並以空格分隔音節
                ipa_syllable_str = f"{initial_ipa}{final_ipa}"
                ipa_sequence.append(ipa_syllable_str)
            except Exception as e:
                # 處理 tl_syllable_to_ipa 失敗的情況 (例如音節不在詞典中)
                print(f"[警告] 轉換音節 '{token}' 失敗。保留為原始 Token。")
                ipa_sequence.append(f"<{token}_FAIL>")

        else:
            # 單獨的數字、標點符號或其他 Token：直接保留
            ipa_sequence.append(token)


    return " ".join(ipa_sequence)


# ----------------------------------------------------
# 主執行流程範例
# ----------------------------------------------------

def main_process_taiwanese_text(raw_text: str):
    print(f"\n--- 原始輸入: {raw_text} ---")

    # 1. 隔離英文專有名詞並初步分詞
    tokens_stage1 = isolate_and_tokenize(raw_text)
    print(f"1. 隔離與分詞: {tokens_stage1}")

    # 2. 應用連讀變調 (並處理連讀組)
    tokens_stage2 = apply_sandhi_and_separate(tokens_stage1)
    print(f"2. 變調與分組: {tokens_stage2}")

    # 3. 最終 G2P 轉換
    final_ipa_output = convert_to_final_ipa(tokens_stage2)

    return final_ipa_output

In [5]:
def generate_metadata(num):
    print(f'generate_metadata({num})')

    df_meta = df.iloc[:num].copy()
    print(f'df.shape:{df.shape}, df_meta.shape:{df_meta.shape}')

    # 音檔
    df_meta['audio_path'] = df_meta['音檔'].apply(lambda item: f"{DATASET_PATH}/{item}".replace(".wav",""))

    # 引號處理
    df_meta['羅馬字'] = df_meta['羅馬字'].apply(lambda item: item.replace("\"",""))
    df_meta['tailo_num'] = df_meta['羅馬字'].apply(lambda item: tailo_num(item))
    # df_meta['tailo_ipa'] =  df_meta['tailo_num'].apply(lambda item: main_process_taiwanese_text(item))

    display(df_meta)

    # audio/speaker1/file001.wav|This is the transcript.|speaker1|neutral
    # 建立 metadata 格式欄位
    # 使用 | 作為分隔符號
    # df_meta['metadata_line'] = (
    #     df_meta['audio_path'] + '|' +
    #     df_meta['tailo_ipa'] + '|' +
    #     'speaker1' + '|' +
    #     'taigi'
    # )

    df_meta['metadata_line'] = (
        df_meta['audio_path'] + '|' +
        df_meta['tailo_num'] + '|' +
        'speaker1' + '|' +
        'taigi'
    )

    # 選擇最終需要匯出的欄位
    metadata_df = df_meta[['metadata_line']]

    metadata_df.to_csv(
        f"{METADATA_OUTPUT_PATH}/metadata_{num}.csv",
        header=False,
        index=False,
        sep='\n'
    )

    metadata_df.to_csv(
        f"{METADATA_OUTPUT_PATH}/metadata_{num}.txt",
        header=False,
        index=False,
        sep='\n'
    )
    display(metadata_df)

# NUM_LIST = [10, 50, 100, 200, 300, 400, 500]

# for i in NUM_LIST:
#     generate_metadata(i)

generate_metadata(100)

,音檔,來源,漢字,羅馬字,長短,audio_path,tailo_num
0,ImTong/SuiSiann_0001.wav,bô_kóng-liáu無講了,彼號……,Hit-lō...,0.92,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,hit4 lo7
1,ImTong/SuiSiann_0002.wav,bô_kóng-liáu無講了,彼號嘿……,Hit-lō--heh...,0.92,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,hit4 lo7 heh4
2,ImTong/SuiSiann_0003.wav,bô_kóng-liáu無講了,我講……,Guá kóng...,1.07,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,gua2 kong2
3,ImTong/SuiSiann_0004.wav,bô_kóng-liáu無講了,哭……你到底是拜幾欲來？,Khàu...Lí tàu-té sī pài-kuí beh lâi?,2.84,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,khau3 li2 tau3 te2 si7 pai3 kui2 beh4 lai5?
4,ImTong/SuiSiann_0005.wav,bô_kóng-liáu無講了,姦……你是咧講啥潲？,Kàn... Lí sī leh kóng siánn-siâu?,2.24,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,kan3 li2 si7 leh4 kong2 siann2 siau5?
...,...,...,...,...,...,...,...
95,ImTong/SuiSiann_0096.wav,ing-gí-sû英語詞,我的電腦怪怪，連袂起去Internet。,"Guá ê tiān-náu kuài-kuài, liân bē khí-lì Inter...",3.41,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,"gua2 e5 tian7 nau2 kuai3 kuai3,lian5 be7 khi2 ..."
96,ImTong/SuiSiann_0097.wav,ing-gí-sû英語詞,你tsa̋ng暝Wi-Fi攏無關乎？,Lí tsa̋ng-mê Wi-Fi lóng bô kuainn--honnh?,2.72,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,li2 tsang9 me5 <ENG>w</ENG>i <ENG>f</ENG>i l...
97,ImTong/SuiSiann_0098.wav,ing-gí-sû英語詞,房間內底收袂著Wi-Fi，你若欲用手機仔可能愛去客廳用。,"Pâng-king lāi-té siu bē tio̍h Wi-Fi, lí nā beh...",6.44,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,pang5 king1 lai7 te2 siu1 be7 tioh8 <ENG>w</E...
98,ImTong/SuiSiann_0099.wav,ing-gí-sû英語詞,伊彼軀衫敢是去Net買--的？,I hit su sann kám sī khì Net bé--ê?,2.84,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...,i1 hit4 su1 sann1 kam2 si7 khi3 <ENG>n</ENG>e...


,metadata_line
0,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
1,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
2,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
3,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
4,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
...,...
95,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
96,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
97,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
98,/content/drive/MyDrive/台語自然語言模型/Dataset/sui-si...
